In [6]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, TimeDistributed, LSTM, Input

In [2]:
# Training Parameters

batch_size = 32
epochs = 5
num_classes = 10

In [3]:
# Embeding Dimensions
rows_hidden = 128
cols_hidden = 128

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


**Normalization**

# Reshapes data to 4D for Hierarchical RNN.

# HRNNs can learn across multiple levels of temporal hierarchy over a complex sequence. Usually, the first recurrent layer of an HRNN encodes a sentence
# (e.g. of word vectors) into a sentence vector. The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to preserve both the word-level and sentence-level structure of the context.

In [5]:
# Reshape Data to 4D for Hierarchical RNN
# Because RNN works on Sequence Based so we have to convert images to 4D

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train /= 255
x_test /= 255

**One Hot Encoding**

In [7]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Creating input conditions for model

row, col, dim = x_train.shape[1:]

x = Input(shape=(row, col, dim))

# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistributed. This wrapper applies a layer to every temporal slice of an input. The input should be at least 3D, and the dimension of index one will be 
# considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

In [9]:
encoded_rows = TimeDistributed(LSTM(rows_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(cols_hidden)(encoded_rows)

In [15]:
encoded_rows.shape

TensorShape([None, 28, 128])

In [10]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [12]:
# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.4447 - accuracy: 0.8507 - val_loss: 0.2313 - val_accuracy: 0.9331
Epoch 2/5
1875/1875 [==============================] - 54s 29ms/step - loss: 0.1071 - accuracy: 0.9671 - val_loss: 0.0701 - val_accuracy: 0.9758
Epoch 3/5
1875/1875 [==============================] - 53s 29ms/step - loss: 0.0760 - accuracy: 0.9768 - val_loss: 0.0557 - val_accuracy: 0.9825
Epoch 4/5
1875/1875 [==============================] - 54s 29ms/step - loss: 0.0606 - accuracy: 0.9819 - val_loss: 0.0483 - val_accuracy: 0.9857
Epoch 5/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0495 - accuracy: 0.9852 - val_loss: 0.0824 - val_accuracy: 0.9762


In [13]:
# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.08241895586252213
Test accuracy: 0.9761999845504761


In [17]:
print(5)

5
